<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Week-02-Friday" data-toc-modified-id="Week-02-Friday-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Week 02 Friday</a></span></li><li><span><a href="#Announcements" data-toc-modified-id="Announcements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Announcements</a></span></li><li><span><a href="#Goals" data-toc-modified-id="Goals-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Goals</a></span></li><li><span><a href="#Using-Timers-in-FSMs" data-toc-modified-id="Using-Timers-in-FSMs-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Using Timers in FSMs</a></span></li><li><span><a href="#Using-Timers-in-FSMs" data-toc-modified-id="Using-Timers-in-FSMs-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Using Timers in FSMs</a></span></li><li><span><a href="#Using-Timers-in-FSMs" data-toc-modified-id="Using-Timers-in-FSMs-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Using Timers in FSMs</a></span></li><li><span><a href="#We-can-also-modify-our-'box'-FSM-from-Wednesday-so-that-the-Romi-always-turns-for-1-second." data-toc-modified-id="We-can-also-modify-our-'box'-FSM-from-Wednesday-so-that-the-Romi-always-turns-for-1-second.-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>We can also modify our 'box' FSM from Wednesday so that the Romi always turns for 1 second.</a></span></li><li><span><a href="#Class-Exercise:-Romi-&quot;Explorer.&quot;" data-toc-modified-id="Class-Exercise:-Romi-&quot;Explorer.&quot;-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Class Exercise: Romi "Explorer."</a></span></li><li><span><a href="#Possible-Solution" data-toc-modified-id="Possible-Solution-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Possible Solution</a></span></li></ul></div>

## Week 02 Friday
Sept 06, 2024

## Announcements
1. A01 (reaction paper) is due **Today**
2. A03 (robot programming 1) is due **Friday, Sept 13**

Four robots have been function-tested (one needs repair and was put away). If you run into any problems collecting data from them or making them move, please let me know ASAP.

## Goals

1. Using "timers" in Finite State Machines
2. Work time for in-class exercise: "Romi Explorer"



## Using Timers in FSMs

Often, we need to keep track of how long an FSM has been in a state, or how long a certain input has been true. We will often want to use an "elapsed time" to trigger a state transition. To do this we need:

1. A "start time" variable in seconds or milliseconds that "saves" the instant that we "turn the timer on" to begin counting, e.g. 
```python
if (notCounting):
    starttime = current_time
```
(note that this stops updating the start time once we *are* counting!)
2. An "elapsed time" variable that keeps track of time *since* the start time, e.g. 
```python 
elapsed=current_time-starttime
```
3. A Boolean variariable that is True when enough time has passed, and False otherwise.
```python
status = elapsed >= some_preset_amount_of_time
```

## Using Timers in FSMs

**The FSM resource on the course website includes an example "timer class" that you can add to the *top* of a webots controller file. Let's talk through how this works.**

In [6]:
%%html
<iframe id="inlineFrameExample" title="Inline Frame Example" width="1000" height="600" src="https://alexanderallenbrown.github.io/ES302_FA24_Students/03_FiniteStateMachines/03_FiniteStateMachines.html#Timers"> </iframe>

## Using Timers in FSMs

**How do we use this in our controller code? First, paste the Timer class somewhere near the top of your controller code. Then, instantiate a timer *before* the while-loop. This timer will have a preset of 1000ms (1 second).**

```python
tmr1 = Timer(1000)
```

**Then, inside the While loop, you can update the timer however you like. Let's see how it behaves if we activate it based on whether the 'k' key is down. The timer's 'update' function needs a $\Delta t$ value in the same units as your preset (milliseconds, in this case)**

```python
while romi.simromi.step(timestep) != -1:
    tmr1.update(keyboard.keyDown()==75,timestep)#uses the romi/webots 'timestep' value defined above (32ms)
    print(tmr1.elapsed,tmr1.state)

```

## We can also modify our 'box' FSM from Wednesday so that the Romi always turns for 1 second.



| **Transition** | **Start** | **Condition** | **End** |
|----------------|-----------|---------------|---------|
|        A        |    F      |    not upk          |  F     |
|         B       |     F     |      upk      |   T    |
|         C       |     T      |     not tmr1.state        |    T    |
|         D       |     T      |      tmr1.state        |    F    |

We have to **update tmr1 in block 1** and use the turn state as the ENABLE input:
```python
while romi.simromi.step(timestep) != -1:
    simtime+=timestep/1000.0
    # romiGoForward(0)
    #BLOCK 1  
    kDown = keyboard.getKey()==75
    UPK = kDown and not kOld
    tmr1.update(T,timestep)
```

<style>
pre[class*=python]
    {font-size: 0.8em;}
</style>
```python
"""romi_test controller."""
#import the sys library so we can get path above
import sys
#add ES302 directory to path because Romi.py is there
sys.path.append("../ES302_Romi")
#now import the Romi library
from Romi import Romi
#import sine function to use
from math import sin

writeData = False

#open a file so we can save data
f = open("data/simdata.txt","w")

# create the Robot instance.
#instantiating a Romi library object in simulation mode
#automatically loads proper WeBots libraries.
#the webots robot class lives inside of romi.simromi
romi = Romi(sim=True)
keyboard = romi.simromi.getKeyboard()
keyboard.enable(32)

class Timer:
    def __init__(self,preset):
        #current "state" of the timer
        self.state = False
        #current elapsed time
        self.elapsed = 0
        #timer will go true if it has counted for more than 1 second
        self.preset = preset
    def update(self,ENABLE,dt):
        #dt is the timestep by which we should count up. make sure its units match your preset!
        #don't set the preset in seconds and increment the elapsed time in milliseconds, for example!
        #ENABLE is a boolean. When it is true, we run up the timer. When it is not, the time resets and we stop counting.
        if(ENABLE):
            #increment time by dt
            self.elapsed+=dt
            self.state=self.elapsed>=self.preset
        else:
            self.elapsed=0
            self.state=False

tmr1 = Timer(1000)

# get the time step of the current world.
timestep = int(romi.simromi.getBasicTimeStep())

#initialize a simulation time for us to use
simtime = 0

def romiTurn(speed):
    romi.update(-speed,speed,90,90,90)

def romiGoForward(speed):
    romi.update(speed,speed,90,90,90)
#initialize old value of k key
kOld = False
#initialize our FSM states (start in one of them!)
F = True
T = False
# Main loop:
while romi.simromi.step(timestep) != -1:
    simtime+=timestep/1000.0
    #BLOCK 1  
    kDown = keyboard.getKey()==75
    UPK = kDown and not kOld
    tmr1.update(T,timestep)
    #Block 2
    A = F and not UPK
    B = F and UPK
    C = T and not tmr1.state
    D = T and tmr1.state
    # Block 3
    F = A or D
    T = C or B
    #Block 4
    if(F):
        romiGoForward(100)
    elif(T):
        romiTurn(100)
    else:
        print("Uh oh")
    kOld = kDown
```

## Class Exercise: Romi "Explorer."

Task: Make the Romi "autononomously explore" its environment using a modified version of our "Box FSM" from Wednesday. 

**Make a copy of your ES302_Webots folder from Wednesday, and rename it Week02_Friday or similar**

Your FSM should make use of at least one timer, and it should make use of the front proximity sensor. It needs no setup, since it is already enabled when your Romi() object is instantiated. In your Controller code's While loop, you can access the front proximity sensor like this:

```python
while romi.simromi.step(timestep) != -1:
    ...#all your code
    proxValNow = romi.proxFrontVal
    #see how it behaves as you approach a wall, to plan how you'll use it:
    print(proxValNow)
```
You may add other behaviors to the Romi other than turn and straight if you wish. I also suggest adding some solids to the Romi's world to see whether it reliably avoids them.

## Possible Solution

| **Transition** | **Start** | **Condition** | **End** |
|----------------|-----------|---------------|---------|
|        A        |    F      |    proxValNow<750          |  F     |
|         B       |     F     |      proxValNow>=750      |   T    |
|         C       |     T      |     not tmr1.state        |    T    |
|         D       |     T      |      tmr1.state        |    F    |

```python
## Full Solution
"""romi_test controller."""
#import the sys library so we can get path above
import sys
#add ES302 directory to path because Romi.py is there
sys.path.append("../ES302_Romi")
#now import the Romi library
from Romi import Romi
#import sine function to use
from math import sin

writeData = False

#open a file so we can save data
f = open("data/simdata.txt","w")

# create the Robot instance.
#instantiating a Romi library object in simulation mode
#automatically loads proper WeBots libraries.
#the webots robot class lives inside of romi.simromi
romi = Romi(sim=True)
keyboard = romi.simromi.getKeyboard()
keyboard.enable(32)

class Timer:
    def __init__(self,preset):
        #current "state" of the timer
        self.state = False
        #current elapsed time
        self.elapsed = 0
        #timer will go true if it has counted for more than 1 second
        self.preset = preset
    def update(self,ENABLE,dt):
        #dt is the timestep by which we should count up. make sure its units match your preset!
        #don't set the preset in seconds and increment the elapsed time in milliseconds, for example!
        #ENABLE is a boolean. When it is true, we run up the timer. When it is not, the time resets and we stop counting.
        if(ENABLE):
            #increment time by dt
            self.elapsed+=dt
            self.state=self.elapsed>=self.preset
        else:
            self.elapsed=0
            self.state=False

tmr1 = Timer(1000)

# get the time step of the current world.
timestep = int(romi.simromi.getBasicTimeStep())

#initialize a simulation time for us to use
simtime = 0

def romiTurn(speed):
    romi.update(-speed,speed,90,90,90)

def romiGoForward(speed):
    romi.update(speed,speed,90,90,90)

kOld = False
F = True
T = False
# Main loop:
# - perform simulation steps until Webots is stopping the controller
while romi.simromi.step(timestep) != -1:
    simtime+=timestep/1000.0
    # romiGoForward(0)
    #BLOCK 1  
    kDown = keyboard.getKey()==75
    UPK = kDown and not kOld
    tmr1.update(T,timestep)
    proxValNow = romi.proxFrontVal
    #Block 2
    A = F and not proxValNow>750
    B = F and proxValNow>750
    C = T and not tmr1.state
    D = T and tmr1.state
    # Block 3
    F = A or D
    T = C or B
    #Block 4
    if(F):
        romiGoForward(100)
    elif(T):
        romiTurn(100)
    else:
        print("Uh oh")
    kOld = kDown
```